# 코나 화재

In [26]:
import time

import pandas as pd
import os
import matplotlib.pyplot as plt
import plotly.express as px
import requests
from bs4 import BeautifulSoup

In [27]:
def filter_by_title_w_keyword(df, keywords):
    temp = []
    for keyword in keywords:
        temp.append(df[df.title.str.contains(keyword)])
    filtered_df = pd.concat(temp, axis=0).drop_duplicates()
    return filtered_df

In [28]:
def filter_by_content_w_keyword(df, keywords):
    temp = []
    for keyword in keywords:
        temp.append(df[df.content.str.contains(keyword).astype(bool).fillna(False)])
    filtered_df = pd.concat(temp, axis=0).drop_duplicates()
    return filtered_df

In [29]:
def filter_by_keyword(df, keywords):
    title_filtered_df = filter_by_title_w_keyword(df, keywords)
    content_filtered_df = filter_by_content_w_keyword(df, keywords)
    filtered_df = pd.concat([title_filtered_df, content_filtered_df], axis=0).drop_duplicates()
    return filtered_df

In [30]:
def extract_day(datetime):
    return datetime.split()[0]

In [31]:
def date_formatting_fmkorea(datetime):
    return datetime.replace('.', '-').strip() + ":00".strip()

In [32]:
def add_created_day_col(df):
    df['created_day'] = df.created_at.map(extract_day)
    return df

In [33]:
def plot_per_day_post_conut_plotly(df, title):
    full_date_daily = pd.date_range(start='2020-01-09', end='2024-06-25', freq='D').to_frame(index=False,
                                                                                             name='full_range_date')
    full_date_daily['full_range_date'] = full_date_daily['full_range_date'].astype(str)

    temp_df = df.sort_values(by=['created_at'])
    per_day_view = temp_df.loc[:, ["created_day", "views"]]
    per_day_view = per_day_view.groupby(per_day_view.created_day)['views'].count().reset_index(name='count')

    full_range_df = pd.merge(full_date_daily, per_day_view, left_on='full_range_date', right_on='created_day',
                             how='left')
    full_range_df = full_range_df.fillna(0)

    # min max norm
    # full_range_df['count'] /= full_range_df['count'].max()

    fig = px.line(full_range_df, x="full_range_date", y="count", title=title)

    y_max = int(full_range_df['count'].max())

    # 세로선 추가 (x=3 위치에 세로선 그리기)
    fig.add_shape(type="line", x0="2020-10-08", x1="2020-10-08", y0=0, y1=y_max,
                  line=dict(color="Red", width=2, dash='dash'))
    fig.add_shape(type="line", x0="2021-03-22", x1="2021-03-22", y0=0, y1=y_max,
                  line=dict(color="Red", width=2, dash='dash'))
    fig.add_shape(type="line", x0="2023-02-28", x1="2023-02-28", y0=0, y1=y_max,
                  line=dict(color="Red", width=2, dash='dash'))
    fig.add_shape(type="line", x0="2023-04-13", x1="2023-04-13", y0=0, y1=y_max,
                  line=dict(color="Red", width=2, dash='dash'))

    fig.add_shape(type="line", x0="2020-10-17", x1="2020-10-17", y0=0, y1=y_max,
                  line=dict(color="#79db93", width=2, dash='dash'))
    fig.add_shape(type="line", x0="2020-09-26", x1="2020-09-26", y0=0, y1=y_max,
                  line=dict(color="#79db93", width=2, dash='dash'))
    fig.add_shape(type="line", x0="2020-10-04", x1="2020-10-04", y0=0, y1=y_max,
                  line=dict(color="#79db93", width=2, dash='dash'))
    fig.add_shape(type="line", x0="2021-01-24", x1="2021-01-24", y0=0, y1=y_max,
                  line=dict(color="#79db93", width=2, dash='dash'))
    fig.add_shape(type="line", x0="2021-06-23", x1="2021-06-23", y0=0, y1=y_max,
                  line=dict(color="#79db93", width=2, dash='dash'))
    fig.add_shape(type="line", x0="2021-07-01", x1="2021-07-01", y0=0, y1=y_max,
                  line=dict(color="#79db93", width=2, dash='dash'))
    fig.add_shape(type="line", x0="2022-01-12", x1="2022-01-12", y0=0, y1=y_max,
                  line=dict(color="#79db93", width=2, dash='dash'))
    fig.add_shape(type="line", x0="2021-02-24", x1="2021-02-24", y0=0, y1=y_max,
                  line=dict(color="#79db93", width=2, dash='dash'))

    # plt.axvline(x="2020-09-26", ymin=0, ymax=1, color='blue', linestyle='dotted') # 코나 화재 (제주)
    # plt.axvline(x="2020-10-04", ymin=0, ymax=1, color='blue', linestyle='dotted') # 코나 화재 (대구)
    # plt.axvline(x="2021-06-23", ymin=0, ymax=1, color='blue', linestyle='dotted') # 코나 화재 (보령)
    # plt.axvline(x="2021-07-01", ymin=0, ymax=1, color='blue', linestyle='dotted') # 코나 화재 (세종)
    # plt.axvline(x="2022-10-17", ymin=0, ymax=1, color='blue', linestyle='dotted') # 코나 화재 (남양주)

    fig.update_layout(
        title=title,
        xaxis_title="X Axis",
        yaxis_title="Y Axis"
    )

    fig.show()
    return full_range_df

In [34]:
def plot_per_day_post_conut_plt(df, title):
    # x축이 될 column 생성
    full_date_daily = pd.date_range(start='2020-01-09', end='2024-06-25', freq='D').to_frame(index=False,
                                                                                             name='full_range_date')
    full_date_daily['full_range_date'] = full_date_daily['full_range_date'].astype(str)

    temp_df = df.sort_values(by=['created_at'])
    per_day_view = temp_df.loc[:, ["created_day", "views"]]
    per_day_view = per_day_view.groupby(per_day_view.created_day)['views'].count().reset_index(name='count')

    full_range_df = pd.merge(full_date_daily, per_day_view, left_on='full_range_date', right_on='created_day',
                             how='left')
    full_range_df = full_range_df.fillna(0)

    plt.figure(figsize=(30, 10))
    plt.plot(full_range_df["full_range_date"], full_range_df['count'])

    plt.axvline(x="2020-10-08", ymin=0, ymax=1, color='red',
                linestyle='dotted')  # 2017년 9월 29일부터 2020년 3월 13일까지 생산된 코나 일렉트릭 25,564대를 리콜
    plt.axvline(x="2021-03-22", ymin=0, ymax=1, color='red', linestyle='dotted')  # 전량 배터리 리콜 결정
    plt.axvline(x="2023-02-28", ymin=0, ymax=1, color='red', linestyle='dotted')  # 코나 EV 2세대 디자인 필름 공개
    plt.axvline(x="2023-04-13", ymin=0, ymax=1, color='red', linestyle='dotted')  # 코나 EV 2세대 정식 출시

    plt.axvline(x="2020-10-17", ymin=0, ymax=1, color='blue', linestyle='dotted')  # 코나 화재 (남양주)
    plt.axvline(x="2020-09-26", ymin=0, ymax=1, color='blue', linestyle='dotted')  # 코나 화재 (제주)
    plt.axvline(x="2020-10-04", ymin=0, ymax=1, color='blue', linestyle='dotted')  # 코나 화재 (대구)
    plt.axvline(x="2021-06-23", ymin=0, ymax=1, color='blue', linestyle='dotted')  # 코나 화재 (보령)
    plt.axvline(x="2021-07-01", ymin=0, ymax=1, color='blue', linestyle='dotted')  # 코나 화재 (세종)

    plt.title(title)
    plt.show()
    return full_range_df

In [35]:
# read csv (posts)
# clien_posts_df = pd.read_csv('data/clien_posts.csv')  
# clien_posts_df['from'] = 'clien'
# fm_korea_posts_df = pd.read_csv('data/fmkorea_posts.csv') 
# fm_korea_posts_df['from'] = 'fm_korea'
# fm_korea_posts_df.created_at = fm_korea_posts_df.created_at.apply(date_formatting_fmkorea)
# 특정 폴더 경로
folder_path = '../resources/코나 화재/naver_cafe/posts'

# 모든 CSV 파일을 저장할 리스트
csv_files = []

# 재귀적으로 폴더 탐색
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.csv'):
            csv_files.append(os.path.join(root, file))
# 모든 CSV 파일을 읽어 DataFrame으로 결합
dataframes = [pd.read_csv(file) for file in csv_files]
naver_cafe_posts_df = pd.concat(dataframes, ignore_index=True)
naver_cafe_posts_df['from'] = 'naver_cafe'

# posts_df = pd.concat([clien_posts_df, fm_korea_posts_df, naver_cafe_posts_df], axis=0)
# assert len(posts_df)==(len(clien_posts_df)+len(fm_korea_posts_df)+len(naver_cafe_posts_df))

In [36]:
keywords = ['코나', '화재']

# clien_posts_df = filter_by_keyword(clien_posts_df, keywords)
# clien_posts_df = add_created_day_col(clien_posts_df)

# fm_korea_posts_df = filter_by_keyword(fm_korea_posts_df, keywords)
# fm_korea_posts_df = add_created_day_col(fm_korea_posts_df)

naver_cafe_posts_df = filter_by_keyword(naver_cafe_posts_df, keywords)
naver_cafe_posts_df = add_created_day_col(naver_cafe_posts_df)
naver_cafe_posts_df = naver_cafe_posts_df.sort_values(by=['created_at'])
# posts_df = filter_by_keyword(posts_df, keywords)
# posts_df = add_created_day_col(posts_df)

# EDA

In [ ]:
from selenium import webdriver

post_count_per_day = naver_cafe_posts_df.groupby('created_day').size()
driver = webdriver.Chrome()
news_count = {} 
for date in post_count_per_day[post_count_per_day >= 3].index:
    url = f"https://www.google.com/search?q=%EC%BD%94%EB%82%98+%ED%99%94%EC%9E%AC+intitle%3A%EC%BD%94%EB%82%98+OR+intitle%3A%ED%99%94%EC%9E%AC&tbs=cdr%3A1%2Ccd_min%3A{"".join(date.split("-"))}%2Ccd_max%3A{"".join(date.split("-"))}&tbm=nws"
    driver.get(url)
    time.sleep(2)
    bs = BeautifulSoup(driver.page_source, 'html.parser')
    news_count[date] = len(bs.find_all('div', attrs={'data-news-cluster-id': True}))
    
print(news_count)
# 날짜별 올라온 뉴스 수 (예시로 임의의 값을 사용, 실제 데이터로 교체)
news_count_per_day = pd.Series(news_count)
# 날짜 범위 생성
date_range = pd.date_range(start=min(post_count_per_day.index.min(), news_count_per_day.index.min()),
                           end=max(post_count_per_day.index.max(), news_count_per_day.index.max()))

# 데이터프레임으로 통합
combined_df = pd.DataFrame(index=date_range)
post_count_per_day.index = pd.to_datetime(post_count_per_day.index)
news_count_per_day.index = pd.to_datetime(news_count_per_day.index)

combined_df['post_count'] = post_count_per_day
combined_df['news_count'] = news_count_per_day

# NaN 값을 0으로 대체
combined_df.fillna(0, inplace=True)

# news_count가 0인 경우 제외
combined_df = combined_df[combined_df['news_count'] > 0]
# 두 값을 나누기z
combined_df['ratio'] = combined_df['post_count'] / combined_df['news_count']  # 0으로 나누기 방지
print(combined_df)

# 2020년 10월 7일 실제로 게시글 확인해보면 그냥 코나 화재 관련해서 지들끼리 싸움나서 게시글이 많음

In [40]:

# Plotly를 사용하여 그래프 시각화
fig = px.line(combined_df, x=combined_df.index, y='ratio',
              title='날짜당 생성된 게시물 수와 뉴스 수의 비율',
              labels={'index': '날짜', 'ratio': '비율'},
              text='ratio')

# 그래프 레이아웃 조정
fig.update_layout(xaxis_tickformat='%Y-%m-%d', xaxis_title='날짜', yaxis_title='비율')
fig.show()



In [ ]:
naver_cafe_posts_df = naver_cafe_posts_df.loc[naver_cafe_posts_df["likes"] > 0]

vis_df = plot_per_day_post_conut_plotly(naver_cafe_posts_df, 'Number of Posts per day (naver cafe)')
print(" \
빨간색 라인 \n\
2018년 4월 12일 코나 EV 출시 \n\
2020년 10월 8일 2017년 9월 29일부터 2020년 3월 13일까지 생산된 코나 일렉트릭 25,564대를 리콜 \n\
2021년 3월 22일 전량 배터리 리콜 결정\n\
2021년 4월 코나 EV 단종\n\
2023년 2월 28일 코나 EV 2세대 디자인 필름 공개\n\
2023년 4월 13일 코나 EV 2세대 정식 출시\
\
파란색 라인 \n\
2020년 9월 26일 코나 화재 제주 \n\
2020년 10월 4일 코나 화재 대구 \n\
2021년 1월 24일 코나 화재 대구  \n\
2022년 10월 17일 코나 화재 남양주  \n\
2020년 6월 23일 코나 화재 보령 \n\
2021년 7월 1일 코나 화재 세종 \n\
")

# news 
# 21. 06. 23 코나 화재 (보령)
# 21. 07. 01 코나 화재 (세종)


In [ ]:
vis_df = plot_per_day_post_conut_plt(naver_cafe_posts_df, 'Number of Posts per day (naver cafe)')
print(
    "2018년 4월 12일 코나 EV 출시 \n\
    2020년 10월 8일 2017년 9월 29일부터 2020년 3월 13일까지 생산된 코나 일렉트릭 25,564대를 리콜 \n\
    2021년 3월 22일 전량 배터리 리콜 결정\n\
    2021년 4월 코나 EV 단종\n\
    2023년 2월 28일 코나 EV 2세대 디자인 필름 공개\n\
    2023년 4월 13일 코나 EV 2세대 정식 출시"
)

In [ ]:
a = """<div id="search"><div data-hveid="CAYQGQ" data-ved="2ahUKEwiYgu2ik-2HAxWUqVYBHbRWAwwQGnoECAYQGQ"><h1 class="bNg8Rb OhScic zsYMMe BBwThe" style="clip:rect(1px,1px,1px,1px);height:1px;overflow:hidden;position:absolute;white-space:nowrap;width:1px;z-index:-1000;-webkit-user-select:none">검색결과</h1><div class="dURPMd" data-async-context="query:%EC%BD%94%EB%82%98%20%ED%99%94%EC%9E%AC%20intitle%3A%EC%BD%94%EB%82%98%20OR%20intitle%3A%ED%99%94%EC%9E%AC" eid="ysq4Zpj1KZTT2roPtK2NYA" id="rso"><div class="MjjYud"><div data-hveid="CAcQAA" data-ved="2ahUKEwiYgu2ik-2HAxWUqVYBHbRWAwwQyIQJKAB6BAgHEAA"><div class="SoaBEf R24aHf" data-hveid="CAgQAA"><div><div data-hveid="CAkQAA" data-news-cluster-id="0" data-news-doc-id="OoN3GU8ws66wBM" data-ved="2ahUKEwiYgu2ik-2HAxWUqVYBHbRWAwwQxPQBKAB6BAgJEAA"><a class="WlydOe" data-ved="2ahUKEwiYgu2ik-2HAxWUqVYBHbRWAwwQxfQBKAB6BAgJEAE" href="https://www.imaeil.com/page/view/2020053116124337257" jsname="YKoRaf"><div class="lSfe4c r5bEn aI5QMe"><div class="gpjNTe"><div class="YEMaTe JFSfwc" style="background-color:#f8f9fa;height:92px;width:92px"><div class="uhHOwf BYbUcd" style="height:92px;width:92px"><img alt="" data-atf="1" data-csiid="ysq4Zpj1KZTT2roPtK2NYA_14" id="dimg_1" src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBwgHBgkIBwgKCgkLDRYPDQwMDRsUFRAWIB0iIiAdHx8kKDQsJCYxJx8fLT0tMTU3Ojo6Iys/RD84QzQ5OjcBCgoKDQwNGg8PGjclHyU3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3N//AABEIAFwAXAMBIgACEQEDEQH/xAAcAAABBQEBAQAAAAAAAAAAAAAFAgMEBgcBCAD/xABDEAACAQMCAgUFCwoHAAAAAAABAgMABBEFEiExBhMiQVEHcYGx0hQyM0JSVGFykZPRFyNDRFOSoaLB0xUWY3ODo/D/xAAaAQACAwEBAAAAAAAAAAAAAAADBAACBQEG/8QAJBEAAgIBBAEEAwAAAAAAAAAAAAECEQMEEiExIhNBUWEFMkL/2gAMAwEAAhEDEQA/AM/FrKxyjOT56I20Myrhy2fOaLWllAIZBIsonyOrKkbQO/I/Cp8VkqqM8c86TllH4YfcrskDydlXP20Kv7G7jY7RIV8VY1b57FQxlVTheYHfU3SdNn1SFxp9qryIuSHkA9dXjJvoHOKRm1xFPbEB3OcZ4MaZZ5CPhHB+saO9Ie3eEMq7oyUODyxQkxqaurFyKHk/aOfTSjJIfjt9tP8AUjupJjA499ESZVjG6QfHf9410mQfHb940sJk4xXzBl96M+euk4I5aRmJMj5HLjXN8vy2+2nioB48/VXdldo4aksagA82p3aRxNT7WxL7WKjbzzzzVgtNGN9HtWPavysVjSzRi6Nx46i3LgqDwvJGdgJJ8KrGqw31vdbYyyhjhkA5itmg0JrFcbVOe8c6Tc6DBfdm4tlcjkxXtD00fHqKdUKZMcZK1IwyfT2IaTjnm2agSAHByK0XpLoU1gWt9jcT3DuqoP0Z1CaRRDEy7mxlxtGa0FS5EabAvfikPwAqXc6fPadbGylivDK8ahIkjMFCMSRyxVosrJVwKi2n3wPnp7bGRjDce/HKmkUhtpBB8DRKyg3OOHOjRgn2ClJpEVNMupm/MW0rDu2xk001nKjMroVYHBBGCK9N6X0hstQQCzE8mAB8ER66reoeTyO+vri7aZVM0jSEeGTmrLb01RW37AXRHD2iFT2SOPfVv0W4lVdkPFc5PDNZFZXt4sbGJwvdsjIK/wAKunQ/X+zm4YxmAZkdyAu3vOfNXnM2jlB74s9BLUxzw2VyaaoDqGrjOkbZkdF85ArH+m/lM91O1toF88ESfposZkPn7hWdXPSTV7hmMmrXLnxLcadSv9TPWClc3R6K1WexmmdzLDI8Y24V1JX6OdC5Y9MuLYi5cKAOAOOFeeX1G8ZmZrhyW5k441augGol9Wis79Emt7qVVYSjgDnGc/8AhVMsci8rGMLhW00e50aynib3MqDed29SOdBYeiiSTAzu24HgAMcKt2r6VZabBI9jAsQhOSoJOQeeKCOVubdSGmy6nisjA58aHD8hte1o6tF6y3xYG13opFbxLcQxLNs5qw5/RVZFu6OZWhVccSqKQBx+mjl30ku7e8bSrl8Tx4PWFeLgjhw5Zxzpg3jmdOu4guMAjAOe6tOOZOmhGWKcU4stPQzpCthazl487doC45k5p+/6eSNct1cYVe4ZNBYpmRpolhRcEZK4xSCICctb7j47R+FEcr5AKNFU06CePgUbB8ONTOkvWafoEjxOY5Lk9TjlwPFuPmH8aPwWeSNjxMfESD+tRelWg3WqaOVQ75ICZERcHfwPZFIevCTo1VpckfJGWwQLKMtcRRNn45xXLuH3MR+fil3d8b7sV1bOWVmVRh1OCj9lh6DRCLonq8ybljgB4dhp13GmrSXIj5N8EWxstQulDWcErBsgYGd3ccfbUNZJYJQwZkkU5BzxBFGr3RdZ0e3zqUF1bxEERsxJQnwGMj0HFBIQJJAGBIz72pVrk7vro1+z6RPreg21ylyXuhGIryPvL4wxI+nn6adS/ih0+Ih3eSOEFwDju9VZbot/dafPJLZtgMAG8OFTrzVri4i2y3BVW98F41lz0L3uujWw66EcavtDmraib/XI7uIYZCCdp5Y+n0Um81MiSPMjEZztLZND4oevuFjsBcTM3PsYOfQTRW56LTRP22ZjgHcPGtCGJRr6M7Jn3X9luOtWogPUSQJI2CSTwBxUZulF7nHW6accARGg/pVP2ncyspQr+0BFSrDX7vT4TDatEqFtxEkCuc+cjPdRXYpwPpqfUDEaxxn6n401edIb6SPq/dDlPBXK+qmkTVGHASyAfLTf6xTy6ZezntaSkn1FZPUaosUU+hh55NVYL/xBR79GJ5k7zzpbayQOyhyO/NGI+iNxce+0vUY/9ohh/MKmWvk4vpjnqLkDuEiEfaRmii9lUfVpJDukUMfEk0PcM0u+FSnfnPKtKTyX6yADDHYjP+sN381SE8leuLxkt4T9PWVKJZm9rDKV2hjjwAzRK3tCP0KMflP2sejlVzn6FXtljd7mBHMNKM0NvbeOxR2uZoVKqTsVxk1RwsJHKkR9LtrQoUu3mRj3oAAPQMGj0dvaqkJt7uYFBtGHzniTxznxqnx6g7uqxIoycAUTivGiAYhSTwPOqbZfJeUoPtF/0XUdLtLXULi+jhu5imIUlxlgo4LyAGSaDXRsZmSWTTYIZHQF41ZMKftqoz31zHHJbTwBw/AHcMbfxqVpdzGLXYyRx7G2gFSTj0V1bujjhCuxEetXSHKYHoqQvSjVE4JIFH0CtLHkr0L51qP3iexXx8lehfOtR+8T2KKBM2/zfrfddsPRST0r15v1+UfVOK0v8lehfOtR+8T2KUPJZoXzrUfvE9ioQyqbX9Zm+E1C4YeG81EbUNSOcX90uTk7ZmHqNbB+SzQvnWo/eJ7FcHks0I/rWo/eJ7FQhjVxf6kIWIuZZ37hKd/roHdWk6r7puGBZjg95zXoE+SzQuXurUfvE9imLryS6BOmxrzUwM90kfsVGdRimn2w6lGBjO7jlwDg1KeI5LSXu4YI2jl6q1pfI7oCIUXUdYCnu66Ph/11weRvo9xP+IavnGPho/7dPQzaaKXhbF3DK/6Mij1ZkUxSBiUUqHHaJ4g8R/Wjml39g9ope4hDcm3EKc+nFaPL5Iej06qJbrU2I5N1qZ+3ZTB8i3RwnJvtXP8AzR/26TbTdhqP/9k="/></div></div></div><div class="SoAPf"><div class="MgUUmf NUnG9d"><g-img class="QyR1Ze ZGomKf" style="margin-right:8px"><img alt="" class="YQ4gaf zr758c" data-atf="1" data-csiid="ysq4Zpj1KZTT2roPtK2NYA_15" height="16" id="dimg_3" src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABAAAAAQCAYAAAAf8/9hAAABgUlEQVQ4jc3SvWsUURQF8N/sbpK1WA07IIpjN4WNhQgWAVHEIoWNTUpRsbIyaUwpVhEFQVBSGQypoiCIhX0QbLSTLRybOIVfk/hRaKJmLfauTv4CPfB49/Heve+ecw//Gkn9UGX5KMYxig2sxVUXY9jEp7QsNrcVqLI8wSlM4zA6WMcK7uEyjuArnuMmHqdl0U+qLG9iFlfQisJfYnUwgqvYwjkcwK94P5dUWT6Jh2jjJ25jHu+wG2dxAcfwEQ9wFN9xuoGZSIZrQeMb9qKJJTzCxbQs3uNSaNHGTAsTkbyO+bQs+lWWL4UWQzTwMuLXWEWOiVaoK1r+XFN9h+0Yib1ZyxlroYeD0fJ4KN3DKzytFViN/QT2RdxrYAF97MJ0leUNg9EdwjPcSMviOparLD+JW0Gpj4WkyvIOljGJH7iLOyHmFF4E7zx0GQr+BFNDI3UxhzP+Ou4+PuA8dtaobGARs2lZrP2xcrS+32DGXbyNX9o4jj0G1l7Bm7QstvwX+A2NrmdHqfzjGAAAAABJRU5ErkJggg==" style="border-radius:2px 2px 2px 2px" width="16"/></g-img><span>매일신문</span></div><div aria-level="3" class="n0jPhd ynAwRc MBeuO nDgy9d" role="heading" style="-webkit-line-clamp:3">엑스코 전자관 주차장서 충전 중인 전기차 화재…차량 전소</div><div class="GI74Re nDgy9d" style="margin-top:8px;-webkit-line-clamp:3">29일 오후 11시 10분쯤 대구시 북구 산격동의 엑스코 전자관 주차장에 세워져 있던 전기자동차에서 불이 났다. 이 화재는 차체를
전소시키고 2천700만원(소방서 추산)...</div><span class="wXEDqf" tabindex="-1">.</span><div class="OSrXXb rbYSKb LfVVr" style="bottom:0px"><span>2020. 5. 31.</span></div></div></div></a></div></div></div></div></div></div></div></div>"""
bs = BeautifulSoup(a,"html.parser")
len(bs.find_all('div', attrs={'data-news-cluster-id': True}))

In [ ]:
# vis_df = plot_per_day_post_conut(fm_korea_posts_df, 'Number of Posts per day (fm korea)')

In [ ]:
# vis_df = plot_per_day_post_conut(naver_cafe_posts_df, 'Number of Posts per day (naver cafe)')

In [ ]:
# vis_df = plot_per_day_post_conut(posts_df, 'Number of Posts per day (total)')